# Thai Wiki Language Model for Text Generation

This notebook details how you can use pretrained language model on [Thai Wikipedia Dump](https://dumps.wikimedia.org/thwiki/latest/thwiki-latest-pages-articles.xml.bz2) to generate texts.

In [0]:
# #uncomment if you are running from google colab
# !pip install sklearn_crfsuite
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
# !pip install fastai
# !pip install emoji

In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [00:31<00:00, 32904377.09it/s]


Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 16327446.42it/s]


In [7]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

#get vocab
thwiki_itos = pickle.load(open(THWIKI_LSTM['itos_fname'],'rb'))
thwiki_vocab = fastai.text.transform.Vocab(thwiki_itos)

#dummy databunch
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=thwiki_vocab, max_vocab=60000, min_freq=3)]
data_lm = (TextList.from_df(dummy_df, imdb, cols=['text'], processor=processor)
    .split_by_rand_pct(0.2)
    .label_for_lm()
    .databunch(bs=64))

data_lm.sanity_check()

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:217: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


In [8]:
#check vocab size
len(data_lm.vocab.itos)

60005

In [9]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load_pretrained(**THWIKI_LSTM)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (800 items)
x: LMTextList
xxbos   xxunk !   meg   ryan   doesn ' t   even   look   her   usual   xxunk   xxunk   self   in   this ,   which   xxunk   makes   me   xxunk   her   xxunk   xxunk   acting   xxunk .   hard   to   believe   she   was   the   producer   on   this   dog .   plus   kevin   xxunk :   what   kind   of   suicide   trip   has   his   career   been   on ?   xxunk xxunk   xxunk xxunk   finally   this   was   directed   by   the   guy   who   did   big   chill ?   must   be   a   replay   of   xxunk   -   hollywood   style .   xxunk !,xxbos   every   once   in   a   long   while   a   movie   will   come   along   that   will   be   so   xxunk   that   i   feel   xxunk   to   xxunk   people .   if   i   labor   all   my   days   and   i   can   save   but   one   soul   from   xxunk   this   movie ,   how   great   will   be   my   joy .   
   
   where   to   begin   my   xxunk   of   pain .   for   xxunk ,   the

In [10]:
print(learn.predict('กาลครั้งหนึ่งนานมาแล้ว ', 200, temperature=0.8, min_p=0.005, sep = ''))

กาลครั้งหนึ่งนานมาแล้ว เขาพบว่า "เราไม่สามารถไปไหนได้" เขาเห็นอะไรบางอย่างอยู่ในบ้านของตน และเชื่อว่าจะเป็นสิ่งที่เขาต้องใช้เวลาในการทำ การที่เขาคิดว่ามันคือการเปลี่ยนผ่าน 
 
 เมื่อเขามีคนบอกให้เขารู้ว่า เขาคิดว่า เขาน่าจะอยู่ในช่วงที่เขาอยู่กับเรา เขาจึงบอกว่า ถ้าเขามี "มันจะไม่เกิดขึ้น" เขาก็จะช่วยโลกให้รอด 
 
 เมื่อเขาไปถึง เขาจึงได้พบกับ "เอลซี" ซึ่งเป็นภรรยาของเขา ขณะที่เขากำลังเดินไปยังเกาะแห่งหนึ่ง โดยที่เขาไม่เคยรู้ว่า เธอเป็นพวกที่ไม่ชอบ แต่จะมาพบกันอีกในครั้งเดียวกัน 
 
 วันหนึ่งที่เขาเดินทางไปยังเกาะแห่งหนึ่ง เขาได้พบกับ "เซลิน" ซึ่งเป็นคนที่เขาได้รู้จัก และได้บอกกับเขาว่า 
